# 1. Import thư viện

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

# 2. Nhập dữ liệu
**Tập dữ liệu đầy đủ**: Bao gồm 26.000.000 đánh giá và 750.000 thẻ được áp dụng cho 45.000 bộ phim bởi 270.000 người dùng. Bao gồm dữ liệu genome thẻ với 12 triệu điểm liên quan trên 1.100 thẻ.

**Tập dữ liệu nhỏ**: Bao gồm 100.000 đánh giá và 1.300 thẻ được áp dụng cho 9.000 bộ phim bởi 700 người dùng.

Chúng ta sẽ xây dựng hệ thống gợi ý đơn giản của mình sử dụng các bộ phim từ tập dữ liệu đầy đủ.

In [2]:
credits = pd.read_csv('input_data/credits.csv')
keywords = pd.read_csv('input_data/keywords.csv')
links_small = pd.read_csv('input_data/links_small.csv')
md = pd.read_csv('input_data/movies_metadata.csv')
ratings = pd.read_csv('input_data/ratings_small.csv')

# 3. Phân tích dữ liệu
## Tệp dữ liệu credits.csv

In [3]:
credits.head()
#credits.iloc[0:3]
# credits['cast'].iloc[0:3]
# credits.iloc[:,0:2]

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [5]:
credits.shape

(45476, 3)

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


**Gồm**: 45476 hàng non-null, 3 cột 'cast', 'crew', 'id'

* **cast**: gồm thông tin về diễn viên chính, dưới dạng json
* **crew**: gồm thông tin về đoàn phim, dưới dạng json
* **id**: id của phim trong metadata

## Tệp dữ liệu keywords.csv

In [7]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
keywords.columns

Index(['id', 'keywords'], dtype='object')

In [9]:
keywords.shape

(46419, 2)

In [10]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


**Gồm**: 46419  hàng non-null, 3 cột 'id', 'keywords'

* **id**: id của phim trong metadata
* **keywords**: gồm thẻ(tag) và keywords cho bộ phim.


## Tệp dữ liệu movies_metadata.csv

In [11]:
md.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
budget,30000000,65000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...


In [12]:
md.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [13]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

**Các Features:**

* **adult:** Đánh giá bộ phim có phù hợp cho trẻ em hay không (dưới 18 tuổi)
* **belongs_to_collection:** Chứa thông tin về bộ (series) của bộ phim, dưới dạng json
* **budget:** Kinh phí của bộ phim, tính bằng đô
* **genres:** Chứa thông tin về thể loại của bộ phim, dưới dạng json
* **homepage:** Trang chính thức của bộ phim
* **id:** ID của bộ phim
* **imdb_id:** ID của phim trong csdl IMBD     
* **original_language:** Ngôn ngữ của phim
* **original_title:** Tựa đề gốc của phim
* **overview:** Mô tả ngắn gọn của phim
* **popularity:** Độ phổ biến, đánh giá của TMDB
* **poster_path:** URL của ảnh bìa phim
* **production_companies:** Chứa thông tin về các công ty tham gia sản xuất, dưới dạng json
* **production_countries:** Chứa thông tin về quốc gia mà bộ phim được quay, dưới dạng json
* **release_date:** Ngày ra mắt được công bố của phim
* **revenue:** Tổng doanh thu của phim
* **runtime:** Thời lượng của phim, tính bằng phút
* **spoken_languages:** Chứa thôn tin về các ngôn ngữ được nói trong phim, dưới dạng json
* **status:** Trạng thái của phim (Released, To Be Released, Announced, etc)
* **tagline:** Tagline của bộ phim
* **title:** Tựa đề chính thức của phim
* **video:** Bộ phim có video giới thiệu trên TMDB hay không
* **vote_average:** Đánh giá trung bình của bộ phim.
* **vote_count:** Số lượng đánh giá của người dùng, trên TMDB

In [14]:
md["spoken_languages"].values

array(["[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}]", ...,
       "[{'iso_639_1': 'en', 'name': 'English'}]", '[]',
       "[{'iso_639_1': 'en', 'name': 'English'}]"], dtype=object)

In [15]:
md.shape

(45466, 24)

## Tệp dữ liệu ratings.csv

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [17]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

* **userId:** ID của người dùng
* **movieId:** ID của phim trong TMDB
* **rating:** Số điểm đánh giá của một người dùng cụ thể
* **timestamp:** Thời điểm thực hiện đánh giá

In [18]:
ratings.shape

(100004, 4)

In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


# 4. Tiền xử lý dữ liệu

* Đến đâu xử lý đến đó

# 5. Xây dựng hệ thống gợi ý (Recommendation System)
# 5.1. Hệ thống gợi ý đơn giản
**Cách thức tiếp cận:**

* Đối với 1 hệ thống đơn giản sẽ đưa ra __gợi ý đơn thuần, tổng quát__ tới tất cả người dùng __dựa trên độ phổ biến của phim và thể loại của phim__. 
* * **Input:** Chọn 1 bộ phim bất kỳ
* * **Xử lý:** Dựa trên độ phổ biến và thể loại của phim
* * **Output:** Cho ra những bộ phim tương tự
* * **Nhược điểm:** Với cùng 1 Input, luôn luôn cho ra kết quả giống nhau, và không sử dụng model, mà thay vào đó, sử dụng __cosine_similarity__. Về cơ bản, __những bộ phim phổ biến hơn và được đánh giá cao hơn sẽ có xác suất cao hơn được khán giả phổ thông yêu thích, và nó không có tính cá nhân hoá cho từng người dùng.__ 

**Thực tế:**

* Việc ứng dụng model này là ở mức vô dùng đơn giản. 
* Chúng ta chỉ đơn giản __sắp xếp các bộ phim dựa trên đánh giá và độ phổ biến__ sau đó đưa ra top những phim đó trong danh sách đã có. 
* Ngoài ra, chúng ta cũng __thêm vào tham số về thể loại để đưa thể loại vào cân nhắc top các phim.__

Chúng ta sẽ xây dựng 1 bảng top 250 bộ phim cho những thể loại cụ thể.

In [20]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval)
md['genres'] = md['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [21]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.946943,21.946943,21.946943


In [22]:
gen_md['genre'].value_counts()


genre
Drama                                    20265
Comedy                                   13182
Thriller                                  7624
Romance                                   6735
Action                                    6596
Horror                                    4673
Crime                                     4307
Documentary                               3932
Adventure                                 3496
Science Fiction                           3049
Family                                    2770
Mystery                                   2467
Fantasy                                   2313
Animation                                 1935
Foreign                                   1622
Music                                     1598
History                                   1398
War                                       1323
Western                                   1042
TV Movie                                   767
Carousel Productions                         1
Vision 

In [23]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


* Chúng ta sử dụng đánh giá của TMDB để tìm ra Top Các phim. 
* Thông qua 1 công thức tính trọng số như sau:



$\large Weighted\; Rating (WR) = (\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
```
Trong đó,
    v số lượng đánh giá (votes) cho phim đó
    m là số lượng vote tối thiểu để bộ phim đạt tiêu chuẩn cho chart
    R là điểm số đánh giá của phim
    C là điểm số đánh giá trung bình cho cả tệp
```

In [24]:
# this is V
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

# this is R
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')

# this is C
C = vote_averages.mean()
C

5.244896612406511

* Tiếp theo, cần tìm ra số "m" hợp lý cho công thức, với m là số lượng đánh giá cần thiết để bộ phim đạt tiêu chuẩn tối thiểu có trong Top.

* Chúng ta sẽ sử dụng phân vị 90% của vote_counts để tìm m. Nghĩa là số m sẽ cao hơn số đánh giá của 95% các bộ phim có trong danh sách.



In [25]:
m = vote_counts.quantile(0.95)
m

434.0

In [26]:
# Tiền xử lý dữ liệu để lấy được ra được thông tin year bằng split bằng '-'

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [27]:
qualified = md[(md['vote_count'] >= m) & 
               (md['vote_count'].notnull()) & 
               (md['vote_average'].notnull())][['title', 
                                                'year', 
                                                'vote_count', 
                                                'vote_average', 
                                                'popularity', 
                                                'genres']]

qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [28]:

s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.946943,21.946943,21.946943


In [29]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

Chúng ta sẽ test thực tế bằng __Top 15 Romance Movies__ 

**Top 15 Romantic Movies**

In [30]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [31]:
build_chart('Animation').head(15)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.909339
5481,Spirited Away,2001,3968,8,41.048867,7.875933
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.772103
2884,Princess Mononoke,1997,2041,8,17.166725,7.771305
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.735274
40251,Your Name.,2016,1030,8,34.461252,7.589820
5553,Grave of the Fireflies,1988,974,8,0.010902,7.570962
19901,Paperman,2012,734,8,7.198633,7.465676
39386,Piper,2016,487,8,11.243161,7.285132
20779,Wolf Children,2012,483,8,10.249498,7.281198


In [32]:
build_chart('Family').head(15)

,title,year,vote_count,vote_average,popularity,wr
1225,Back to the Future,1985,6239,8,25.778509,7.893053
359,The Lion King,1994,5520,8,21.605761,7.879754
5481,Spirited Away,2001,3968,8,41.048867,7.835635
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.650968
926,It's a Wonderful Life,1946,1103,8,15.031588,7.490637
19901,Paperman,2012,734,8,7.198633,7.301918
39386,Piper,2016,487,8,11.243161,7.071694
20779,Wolf Children,2012,483,8,10.249498,7.066709
31658,Feast,2014,420,8,7.365663,6.980490
25044,Song of the Sea,2014,420,8,6.967358,6.980490


In [33]:
build_chart('Action').head(15)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.955099
12481,The Dark Knight,2008,12269,8,123.167259,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.918382
256,Star Wars,1977,6778,8,42.149697,7.908327
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.896841
4135,Scarface,1983,3017,8,11.299673,7.802046
9430,Oldboy,2003,2000,8,10.616859,7.711649
1910,Seven Samurai,1954,892,8,15.01777,7.426145


In [34]:
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917
